In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
path = ''
import gc
#from sklearn.cross_validation import train_test_split 
from sklearn.metrics import mean_squared_error as rmse
from sklearn.cross_validation import KFold
import sys
from data_load import loadHDF_train
from data_load import loadHDF_test
from time import time
from sys import stdout

In [2]:
colnames = [u'timestamp', u'time_hr', u'meanInv.custid', u'q75Inv.custid', u'q95Inv.custid',
            u'q999Inv.custid', u'rInvGT10.custid', u'rInvGT2.custid',
            u'meanInv.curHr', u'q75Inv.curHr', u'q95Inv.curHr', u'q999Inv.curHr',
            u'rInvGT10.curHr', u'rInvGT2.curHr', u'meanInv.pltfHr',
            u'q75Inv.pltfHr', u'q95Inv.pltfHr', u'q999Inv.pltfHr',
            u'rInvGT10.pltfHr', u'rInvGT2.pltfHr', u'meanInv.grpidHr',
            u'q75Inv.grpidHr', u'q95Inv.grpidHr', u'q999Inv.grpidHr',
            u'rInvGT10.grpidHr', u'rInvGT2.grpidHr']
datafilehead = path + 'feat_1hr_'
timelimit = 3600.*24*np.array([153,183]) # last 30 days 
seed = 555#555
np.random.seed(seed)
np.random.randint(0,1000)

410

In [3]:
def preprocess(d, timelimit=[0,999999], missing=0, index = []):
    if(len(index) > 0):
        index = index[(d['timestamp'] >= timelimit[0]) & (d['timestamp'] < timelimit[1])]
        d = d[(d['timestamp'] >= timelimit[0]) & (d['timestamp'] < timelimit[1])]

    d.drop('timestamp', inplace=True, axis=1)
    d.fillna(missing, inplace=True)
    if(len(index) == 0):
        return d
    else:
        return d, index

In [4]:
dtr, pred, dtr_index = loadHDF_train(datafilehead+'train.h5', key='train', 
                                     selection = 'index > {0}'.format(35730865), columns = colnames)

In [5]:
## preprocess
dtr, dtr_index = preprocess(dtr, timelimit = timelimit, missing=0, index=dtr_index)
pred = preprocess(pred, timelimit, missing=0)

In [6]:
dte, dte_index = loadHDF_test(datafilehead+'test.h5', key='test', columns = colnames)
dte = preprocess(dte, missing=0)
num_te = dte.shape[0]

In [7]:
print pred

            invest
index             
36530990  1.055579
35917280  1.041684
36540585  1.160324
35900844  2.202429
36231867  1.427530
36198698  1.080162
35976722  1.456923
36201019  2.603239
36566821  1.375158
35754774  1.084170
36119887  1.080162
36239744  1.272551
36501423  1.166737
35877624  1.032065
36287535  1.032065
36324305  1.083368
36080635  1.024049
36418941  1.480972
36584579  1.176356
36652564  1.277895
36295470  1.400810
35902780  1.080162
35790767  1.240486
36089832  1.277895
35794370  1.240486
35796717  9.016194
36078640  2.389474
36255780  1.120243
35992172  2.523077
35846049  1.048097
...            ...
36761129  1.801619
37449517  1.040081
40002545  1.055579
38483347  1.111158
36984843  1.400810
38424665  1.160324
38028229  1.024049
37444975  1.008016
39868324  1.277895
36987139  1.240486
38796018  1.444632
36935941  1.080162
37897685  1.080162
38930131  1.027789
39667684  1.194526
40702663  1.400810
37793848  1.833684
38661048  1.032065
38227780  1.055579
39754818  1.

In [8]:
X = dtr 
X_test = dte
y = pred['invest'].values 

In [9]:
# Import for Linear Regression
import sklearn
from sklearn.linear_model import LinearRegression

# Create a LinearRegression Object
lreg = LinearRegression()

# Implement Linear Regression
lreg.fit(X,y)

print ' The estimated intercept coefficient is %.2f ' %lreg.intercept_
print ' The number of coefficients used was %d ' % len(lreg.coef_)

# # Set a DataFrame from the Features
# coeff_df = DataFrame(df_train.columns)
# coeff_df.columns = ['Features']

# # Set a new column lining up the coefficients from the linear regression
# coeff_df["Coefficient Estimate"] = pd.Series(lreg.coef_)

# # Show
# coeff_df 

 The estimated intercept coefficient is 0.07 
 The number of coefficients used was 25 


In [10]:
## Predictions on training and testing sets
pred_test = lreg.predict(X_test) 
print pred_test

[ 1.11985745  1.11985745  1.11985745 ...,  1.20440287  1.20440287
  1.20440287]


In [11]:
print len(pred_test)
df_submission = DataFrame({'prediction':pred_test})
submission_4 = df_submission['prediction']
submission_4.to_csv('submission_4.csv', index = False)

4606743
